# SPECTRE TEST
- Using H5 Model trained using ISCX-IDS-2012 Dataset
- Testing using Malware Capture Facility Project Dataset - CTU-Malware-Capture-Botnet-135-1 Stlrat DDoS (MD5: `c7838b75ba10b0341554d25fbcc3bbc0`)

## Import Dependencies

In [3]:
from scapy.all import *
from scapy.layers.inet import *
import h5py
import matplotlib.pyplot as plt
import numpy as np

In [4]:
print(f"Scapy Version: {scapy.__version__}")
print(f"Python {sys.version}")
print(f"Scapy Version: {np.__version__}")

Scapy Version: 2.5.0.dev68
Python 3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]
Scapy Version: 1.23.5


## Load H5 Model

In [5]:
import h5py

try:
    # Load the H5 model
    with h5py.File('/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/FYP_Finalh5.h5', 'r') as file:
        # Print metadata
        print("H5 file metadata:")
        print("==================")
        for key, value in file.attrs.items():
            print(f"{key}: {value}")
        print("==================")
        print("Done!")
except:
    print("Error loading H5 file.")

H5 file metadata:
backend: tensorflow
keras_version: 2.12.0
model_config: {"class_name": "Functional", "config": {"name": "model_1", "trainable": true, "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 50, 50, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_2"}, "name": "input_2", "inbound_nodes": []}, {"class_name": "Conv2D", "config": {"name": "block1_conv1", "trainable": false, "dtype": "float32", "filters": 64, "kernel_size": [3, 3], "strides": [1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "block1_conv1", "inbound_nodes": [[["input_2", 0, 0, {}]]]}, {"class_na

## Load the pcap file

In [6]:
# ISCX-IDS-2012 - 14 JUN
#packets = scapy.rdpcap('/mnt/Data/SPECTRE/Dataset/ISCX-2012/PCAP/testbed-14jun.pcap') 

# Malware Capture Facility Project - CTU-Malware-Capture-Botnet-135-1 Stlrat DDoS
packets = scapy.utils.rdpcap('/mnt/Data/SPECTRE/Dataset/Malware Capture Facility Project/2015-09-10_winlinux.pcap') 

## Extract the features from each packet, preprocess the data, and store it in a list:

In [7]:
data = []
for packet in packets:
    # Extract the required information from each packet
    src_ip = packet[scapy.IP].src
    dst_ip = packet[scapy.IP].dst
    src_port = packet[scapy.TCP].sport
    dst_port = packet[scapy.TCP].dport
    protocol = packet[scapy.IP].proto
    
    # Preprocess the data and store it in a list
    processed_data = [src_ip, dst_ip, src_port, dst_port, protocol]
    data.append(processed_data)


AttributeError: module 'scapy' has no attribute 'IP'

## Prepare the data for the model

In [ ]:
data = np.array(data)
data = data.reshape(data.shape[0], data.shape[1], 1)

## Use the model to predict anomalies

In [ ]:
predictions = model.predict(data)

## Calculate the detection rate and amount of anomalies detected

In [ ]:
detection_rate = np.sum(predictions)/len(predictions)
anomalies_detected = np.sum(predictions)

# Results

In [ ]:
x = ['Detection rate', 'Anomalies detected']
y = [detection_rate, anomalies_detected]

plt.bar(x, y)
plt.show()